In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)
# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)


Toggle code

In [2]:
# all modules necessary for this nb
import os
import sys
import pickle

import numpy as np
import pylab as pl
from sklearn.covariance import EmpiricalCovariance
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.metrics import silhouette_score as clust_score
from sklearn.preprocessing import StandardScaler
from scipy import stats as sstats

# setting parameters for default matplotlib plots
%matplotlib inline

In [3]:
pl.style.use('fabiostyle')
from ipywidgets import interact

# needs to find the library of functions
sys.path.append('../../../../code/')  # to be replaced!

import utils as ut
import plots as pt

In [4]:
# a double percentage sign indicates a magic function. in this case, now we are writing this cell in javascript.

In [5]:
NOTEBOOK_NAME = 'preprocessing'

In [6]:
from pickleshare import PickleShareDB

autorestore_folder = os.path.join(os.getcwd(), 'autorestore', NOTEBOOK_NAME)
db = PickleShareDB(autorestore_folder)
import sys
from workspace import *
import IPython
ip = IPython.get_ipython()

# this will restore all the saved variables. ignore the errors listed.
load_workspace(ip, db)

# use `save_worspace(db)` to save variables at the end

In [7]:
data_folder = '../data'

In [8]:
traces = np.loadtxt(os.path.join(data_folder, 'C.txt')).T #denoised traces
traces_raw = np.loadtxt(os.path.join(data_folder, 'C_raw.txt')).T
areas = ut.load_spatial_footprints_A(os.path.join(data_folder, 'A.txt'))
events = np.loadtxt(os.path.join(data_folder, 'S.txt')).T
# dff = np.loadtxt(os.path.join(data_folder, 'C_df.txt')).T
# mean_image, contours = ut.load_spatial_footprints(os.path.join(data_folder, 'Coor.mat'),
#                                                   os.path.join(data_folder, 'Cnn.txt'),
#                                                   key='coor')

In [9]:
filename = os.path.join(data_folder, 'behavior_codes.txt')
behavior = ut.read_behavior(filename)
events_list = np.unique([b[1] for b in behavior])

In [10]:
# grab time axis from the xml file

import xml.etree.ElementTree as ET
xmlfile = os.path.join(data_folder, 'tseries.xml')
print "I infer the time axis from:\n", xmlfile
tree = ET.parse(xmlfile)
root = tree.getroot()

# unfortunately we miss the first frame
time_ax = np.r_[[child.attrib['absoluteTime']
                 for child in root.iter('Frame')]].astype(float)



I infer the time axis from:
../data/tseries.xml


In [13]:
# sync times
start_2p = ut.parse_behavior(behavior, 'BEGIN')[0]
behavior = [[float(b[0])-start_2p, b[1]] for b in behavior]
time_ax -= time_ax[0]

In [14]:
print len(traces)
print len(time_ax)

8520
17040


In [15]:
ratio = int(np.floor(time_ax.shape[0]/traces.shape[0]))
print ratio

2


In [16]:
time_ax = time_ax[::ratio] # use this if video was averaged and need to adjust xml output to match

In [17]:
print len(traces)
print len(time_ax)

8520
8520


In [18]:
time_ax = time_ax[0:len(traces)] # use this if any video frames were truncated (often need to do this a video is averaged)

In [19]:
print len(traces)
print len(time_ax)

8520
8520


In [20]:
# make sure presentations are correct in timing.
#b = [b[1] for b in behavior]
#print b

        

In [21]:
behavior = ut.sync_behavior_to_xml(time_ax, behavior)

In [22]:
# -----------------------------------------------------------
# these times are relative to the single cycle
# and centered around tone onset
CYCLE_START = -5  # seconds
CS_DURATION = 2  # seconds
DELAY = 2  # seconds
AFTER_DELAY_DURATION = 4  # seconds
US_START = 4
CYCLE_DURATION = abs(CYCLE_START) + CS_DURATION + DELAY + AFTER_DELAY_DURATION

# -----------------------------------------------------------
# these times are absolute times, taken from the arduino file
# when the tones starts and ends
odor_CSm_ons = ut.parse_behavior(behavior, 'TONE_CSM')
odor_CSm_offs = ut.parse_behavior(behavior, 'TONE_CSM', offset=CS_DURATION)
odor_rw_ons = ut.parse_behavior(behavior, 'TONE_RW')
odor_rw_offs = ut.parse_behavior(behavior, 'TONE_RW', offset=CS_DURATION)
rewards = np.r_[ut.parse_behavior(behavior, 'REWARD')]
licks = np.r_[ut.parse_behavior(behavior, 'LICK')]

# -----------------------------------------------------------
# when the experiment starts and ends, in absolute time
cycles_starts = ut.parse_behavior(behavior, '^TONE_*', offset=CYCLE_START) #looks for arduino line that begins w/ 'TONE'
cycles_ends = ut.parse_behavior(behavior, '^TONE_*', offset=CYCLE_DURATION+CYCLE_START)

cycle_subtract = 0   #do we need to subtract off the last cycle because it's too short???
if cycle_subtract !=0:
    cycles = np.r_[zip(cycles_starts,  # offset will be ADDED, with sign
                   cycles_ends)][:cycle_subtract]
else:
    cycles = np.r_[zip(cycles_starts,  # offset will be ADDED, with sign
                   cycles_ends)]
print 'we are subtracting off this many cycles'
print cycle_subtract
# -----------------------------------------------------------
# which trials are a.p. and which reward
is_rewardt = [any(map(lambda t: (t>=s) and (t<e), odor_rw_ons)) for s, e in zip(cycles_starts, cycles_ends)]
is_CSmt = [any(map(lambda t: (t>=s) and (t<e), odor_CSm_ons)) for s, e in zip(cycles_starts, cycles_ends)]


we are subtracting off this many cycles
0


In [21]:
is_rewarded = []
for s, e in cycles:
# s, e = cycles[np.where(is_rewardt)[0][i]]
    try:
        r = rewards[(rewards>=s)*(rewards<e)][0]
        later_licks = licks-r
        is_rewarded.append((np.where((later_licks>=0) * (later_licks<2))[0][0])>0)
    except IndexError:
        is_rewarded.append(False)

In [22]:
licks = np.r_[ut.parse_behavior(behavior, 'LICK')]
licks_bs = 1.*ut.compute_licks_during(licks, cycles,
                                      start=-CYCLE_START-DELAY,
                                      end=-CYCLE_START)  # w.r.t. cycle start
licks_cs = 1.*ut.compute_licks_during(licks, cycles,
                                      start=-CYCLE_START,
                                      end=-CYCLE_START+CS_DURATION)
licks_tc = 1.*ut.compute_licks_during(licks, cycles,
                                      start=-CYCLE_START+CS_DURATION,
                                         end=-CYCLE_START+CS_DURATION+DELAY)
licks_cs_tc = 1.*ut.compute_licks_during(licks, cycles,
                                         start=-CYCLE_START,
                                         end=-CYCLE_START+CS_DURATION+DELAY)
licks_tc_us = 1.*ut.compute_licks_during(licks, cycles,
                                         start=-CYCLE_START+CS_DURATION,
                                         end=-CYCLE_START+CS_DURATION+DELAY+AFTER_DELAY_DURATION)
licks_cs_tc_us = 1.*ut.compute_licks_during(licks, cycles,
                                      start=-CYCLE_START,
                                      end=-CYCLE_START+CS_DURATION+DELAY+AFTER_DELAY_DURATION)
lickrates_bs = 1.*licks_bs/(DELAY+AFTER_DELAY_DURATION)
lick_ratios = np.nan_to_num(1.*(licks_tc_us-licks_bs)/(licks_tc_us+licks_bs))
lick_di = np.nan_to_num(1.*(np.mean(licks_tc[is_rewardt]-licks_bs[is_rewardt]) -
                            np.mean(licks_tc[is_CSmt]-licks_bs[is_CSmt]))/np.sqrt(0.5*(np.std(licks_tc)**2+np.std(licks_bs)**2)))
    
good_lick_trials = (licks_bs+licks_tc_us) >= 5

is_errCSmt = (lick_ratios>0.8) * ((licks_tc_us+licks_bs) > 4) * is_CSmt
print is_errCSmt.sum()

is_corrCSmt = ((licks_tc_us)==0) * is_CSmt
print is_corrCSmt.sum()
print licks_bs

1
41
[ 0.  0.  0.  0.  4.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  1.  3.  0.  0.  0.  0.  0.  3.  1.  0.  0.  0.  1.  0.  0.
  0.  0.  0.  0.  2.  0.  0.  1.  0.  2.  0.  2.  0.  0.  0.  1.  0.  0.
  0.  0.  0.  1.  0.  1.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  3.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  2.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  5.  1.  0.  0.  2.  0.]


/home/fabios/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:21: RuntimeWarning: invalid value encountered in divide


In [23]:
time_ax_single = ut.extract_single_cycle_time_ax(time_ax, cycles,
                                                 cycle_duration=CYCLE_DURATION, cycle_start=CYCLE_START)

In [24]:
save_workspace(db)

Could not store variable 'ET'. Skipping...
Could not store variable 'di'. Skipping...
Could not store variable 'os'. Skipping...
Could not store variable 'IPython'. Skipping...
Could not store variable 'pt'. Skipping...
Could not store variable 'sstats'. Skipping...
Could not store variable 'pl'. Skipping...
Could not store variable 'ut'. Skipping...
Could not store variable 'pickle'. Skipping...
Could not store variable 'ip'. Skipping...
Could not store variable 'np'. Skipping...
Could not store variable 'sys'. Skipping...
